In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from datetime import datetime
pd.set_option("display.max_colwidth",50)

In [2]:
Start = "https://www.presseportal.de/blaulicht/nr/4970"
html = urlopen(Start)
bs = BeautifulSoup(html, 'html.parser')
Links=bs.find_all("h2", class_="news-headline-clamp" )

In [3]:
#Try Pages not found again
def getPagesNotfound(AllPagesNotfound):
    """ Get all pages that threw an error"""
    LinksNotFoundBefore=[]
    LinksStillNotFOund=[]
    for SinglePage in AllPagesNotfound:
        try:
            bs = BeautifulSoup(SinglePage, 'html.parser')
            Tag=bs.find(class_="pagination-next")
            
            LinksNotFoundBefore.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            time.sleep(.3)
        except:
            time.sleep(1)
            LinksStillNotFOund.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
    return LinksNotFoundBefore, LinksStillNotFOund
 
    
#getallsites
Start = "https://www.presseportal.de/blaulicht/nr/4970"
def getallsites(Start, MaxP):
    """Get the links of MaxP amount of subsequent sites that follow "Start" which contain the links to the individual news articles."""
    Pages=[]
    PagesNotfound=[]
    html = urlopen(Start)
    Success =0
    NoSuccess=0
    for i in range(1,MaxP+1):
        try:
            bs = BeautifulSoup(html, 'html.parser')
            Tag=bs.find(class_="pagination-next")
            Pages.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            html = urlopen("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            time.sleep(.3)
            Success +=1
        except:
            time.sleep(1)
            NoSuccess +=1
            PagesNotfound.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
        time.sleep(.3)
        print("getallsites Progress "+str(round(((i/MaxP)*100),2))+"%" + " Success: " +str(Success) + " NoSuccess: "+str(NoSuccess), flush=True,end='\r')
    LinksNotFoundBefore,LinksStillNotFOund = getPagesNotfound(PagesNotfound)
    
    for Link in LinksNotFoundBefore:
        Pages.append(Link)
    
    return Pages,LinksStillNotFOund


In [4]:
AllSites, SitesNotFound =getallsites(Start,1)

In [5]:
def getCorpusFromLink(AllSites):
    """Get the corpus for each given link"""
    MainsiteCorpus=[]
    RemainingSites= []
    for Site in AllSites:
        try:
            html = urlopen(Site)
            bs = BeautifulSoup(html, 'html.parser')
            Links=bs.find_all("h3", class_="news-headline-clamp" )
            MainsiteCorpus.append(Links)
        except:
            RemainingSites.append(Site)
            
    return MainsiteCorpus, RemainingSites

def getLinksFromCorpus(MainsiteCorpus):
    """ectract the link to each news for a given corpus"""
    Linklist =[]
    for Site in MainsiteCorpus:
        for Headline in Site:
            Article= (Headline.findChild("")["href"])
            Linklist.append(Article)
    return Linklist

def findMainArticleForGivenText(Corpus):
    """find the main article for a given news-corpus"""
    s = ''
    for element in Corpus:
        s += '\n' + ''.join(element.findAll(text = True))
        s
    start = 'Polizeipräsidium Frankfurt am Main'
    end = 'Rückfragen bitte an:'
    s = (s[s.find(start)+len(start):s.rfind(end)])
    return s

In [6]:
def getNewsFromLinks(Liste):
    """Return the Headline, Main Article, Link and the Pages not found for each given link"""
    counter=0
    counter2=0
    News=[]
    Headline=[]
    Hauptartikel=[]
    LinkNotfound=[]
    List = Liste[1:3]
    for Link in Liste:
        try:
            html=urlopen('https://www.presseportal.de{}'.format(Link))
            bs = BeautifulSoup(html, 'html.parser')
            Ueberschrift= bs.find("h1").get_text()
            WholeText =bs.find_all('p')
            MainArticle =findMainArticleForGivenText(WholeText)
            Headline.append(Ueberschrift)
            Hauptartikel.append(MainArticle)
            counter+=1
            News.append('https://www.presseportal.de{}'.format(Link))
            time.sleep(.3)
            print("found-> "+ str(counter) + "/"+ str(counter2)+ " of "+ str(len(Liste)),flush=True,end='\r')
        except:
            counter2+=1
            LinkNotfound.append(Link)
            print("found-> "+ str(counter) + "/"+ str(counter2)+ " of "+ str(len(Liste)),flush=True,end='\r')
        
    return Headline,Hauptartikel,News,LinkNotfound

In [7]:
Corpus, Other =getCorpusFromLink(AllSites)

In [8]:
LinksForAllNews

NameError: name 'LinksForAllNews' is not defined

In [9]:
LinksForAllNews = getLinksFromCorpus(Corpus)

In [10]:
AlleHeadlines, AlleHauptartikel, AlleWebsiten, LinkNotfound =getNewsFromLinks(LinksForAllNews)

In [12]:
df = pd.DataFrame({'Headline':AlleHeadlines,'Hauptartikel':AlleHauptartikel,'Link':AlleWebsiten})

In [13]:
df

,Headline,Hauptartikel,Link
0,POL-F: 190422 - 454 Frankfurt-Sachsenhausen: D...,\n\nFrankfurt (ots)\n (em) In der Nacht von Ka...,https://www.presseportal.de/blaulicht/pm/4970/...
1,POL-F: 190422 - 453 Frankfurt-Gallus: Brutale ...,\n\nFrankfurt (ots)\n (em) Am Ostersonntag (21...,https://www.presseportal.de/blaulicht/pm/4970/...
2,POL-F: 190422 - 452 Frankfurt-Westend: Festnah...,\n\nFrankfurt (ots)\n (em) Am Karfreitag (19.0...,https://www.presseportal.de/blaulicht/pm/4970/...
3,POL-F: 190422 - 451 Frankfurt-Bornheim: Volksf...,\n\nFrankfurt (ots)\n (em) In der Samstagnacht...,https://www.presseportal.de/blaulicht/pm/4970/...
4,POL-F: 190422 - 450 Frankfurt-Sachsenhausen: F...,\n\nFrankfurt (ots)\n (em) In der Nacht von So...,https://www.presseportal.de/blaulicht/pm/4970/...
5,POL-F: 190422 - 449 Frankfurt-BAB 648: Motorra...,\n\nFrankfurt (ots)\n (em) Am Samstagnachmitta...,https://www.presseportal.de/blaulicht/pm/4970/...
6,POL-F: 190420 - 448 Frankfurt-BAB3: Motorradfa...,\n\nFrankfurt (ots)\n (em) Heute Vormittag (20...,https://www.presseportal.de/blaulicht/pm/4970/...
7,"POL-F: 190420 - 447 Frankfurt: ""Car-Freitag"" -...",\n\nFrankfurt (ots)\n (em) Die Kontrolleinheit...,https://www.presseportal.de/blaulicht/pm/4970/...
8,POL-F: 190419 - 446 Frankfurt-Gutleutviertel: ...,\n\nFrankfurt (ots)\n (em) In der vergangenen ...,https://www.presseportal.de/blaulicht/pm/4970/...
9,POL-F: 190418 - 445: Oberaurach/Landkreis Hass...,\n\nFrankfurt (ots)\n Seit vergangenem Freitag...,https://www.presseportal.de/blaulicht/pm/4970/...


In [63]:
df.head()

,Headline,Hauptartikel,Link
0,POL-F: 190418 - 444 Frankfurt-Griesheim: Unfal...,\n\nFrankfurt (ots)\n (ka) Heute Morgen stieße...,https://www.presseportal.de/blaulicht/pm/4970/...
1,POL-F: 190418 - 443 Frankfurt-Stadtgebiet: Ges...,\n\nFrankfurt (ots)\n (fue) Auch in der kommen...,https://www.presseportal.de/blaulicht/pm/4970/...
2,POL-F: 190418 - 442 Frankfurt-Oberrad: Wohnung...,\n\nFrankfurt (ots)\n (fue) Die 68-jährige Mie...,https://www.presseportal.de/blaulicht/pm/4970/...
3,POL-F: 190418 - 441 Bundesautobahn 5: Zeugensuche,"\n\nFrankfurt (ots)\n (fue) Am Mittwoch, den 1...",https://www.presseportal.de/blaulicht/pm/4970/...
4,POL-F: 190417 - 440 Frankfurt-Innenstadt: Fest...,\n\nFrankfurt (ots)\n (em) Gestern (16.04.2019...,https://www.presseportal.de/blaulicht/pm/4970/...


In [64]:
#Replace escape sharacters
df['Headline']= df["Headline"].apply(lambda st: st.replace("\t"," "))
df['Headline']= df["Headline"].apply(lambda st: st.replace("\n"," "))
df['Hauptartikel']= df["Hauptartikel"].apply(lambda st: st.replace("\n"," "))

df["date"] =df.Headline.str.extract('(\d+)')
df['date'] = df['date'].apply(lambda x: "{}{}".format('20', x))
#ffill if date is not found
df['date'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d') if len(x)==8 else "").fillna(method='ffill')
df['Headline'] = df['Headline'].str.replace('\d+', '')

In [65]:
#Extract case
#df["case"] =df.Headline.str.extract('(\s\d{3,4}\s)')
#df["case"] = df["case"].astype(str)

In [66]:
#Find index of Case in Headline
#df['col3'] = [i2.index(i1) for i1,i2 in zip(df.case,df.Headline)]
#COnvert into numeric
#df['col3'] = pd.to_numeric(df['col3'] )
#Remove beginning of Headline
#df['Headline'] = [A[B:] for A, B in zip(df["Headline"], df["col3"])]

In [67]:
#Strip whitespaces
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())
#Remove beginning from Headline (remove 4digit Number)
df['Headline']=df['Headline'].str.replace("\d{4}","",regex=True)
df['Headline']=df['Headline'].str.replace("\d{3}","",regex=True)
df['Headline']=df['Headline'].str.replace("POL-F:","")

#Strip whitespaces again
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())

In [68]:
#Function to find nth substring
def find_nth(s, substr, n):
    i = 0
    while n >= 0:
        n -= 1
        i = s.find(substr, i + 1)
    return i

In [69]:
#Extract Location
df['Location']= df["Headline"].apply(lambda st: st[0:st.find(":")])
#remove Stuff from Location
df['Location']=df['Location'].str.replace("/"," ")
df['Location']=df['Location'].str.replace("Frankfurt","")
df['Location']=df['Location'].str.replace("-"," ")
df['Location']= df["Location"].apply(lambda st: st.lstrip())
#Remove Headline
df['Headline']= df["Headline"].apply(lambda st: st[st.find(":")+1:])
#Strip whitespace again
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())
#Extract second location
df['Location2']= df['Location'].str.split(' ').str[1]
#Clean Location
df['Location']= df['Location'].str.split(' ').str[0]
#Remove start of Hauptartikel e.g. Frankfurt (ots) - (ka)
df["Headline"].apply(lambda st: st[st.find(" - ")+2:find_nth(st," ",3)])
#Extract start of Hauptartikel
df["Beschreibung"]= df["Hauptartikel"].apply(lambda st: st[0:find_nth(st,")",1)+1])
#Remove start from Hauptartikel
df["Hauptartikel"]= df["Hauptartikel"].apply(lambda st: st[find_nth(st,")",1)+1:])

In [70]:
df=df.drop_duplicates()
df.count()

Headline        539
Hauptartikel    539
Link            539
date            539
Location        539
Location2       102
Beschreibung    539
dtype: int64

In [72]:
df.to_csv("Test.csv",sep="@",encoding="UTF-16")

In [75]:
df.head()

,Headline,Hauptartikel,Link,date,Location,Location2,Beschreibung
0,Unfall zwischen Auto und Straßenbahn,Heute Morgen stießen auf der Mainzer Landstra...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Griesheim,NaN,Frankfurt (ots) (ka)
1,Geschwindigkeitsmessungen,Auch in der kommenden Woche werden im Frankfu...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Stadtgebiet,NaN,Frankfurt (ots) (fue)
2,Wohnungsbrand,Die 68-jährige Mieterin einer Wohnung in der ...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Oberrad,NaN,Frankfurt (ots) (fue)
3,Zeugensuche,"Am Mittwoch, den 17. April 2019, gegen 22.10 ...",https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-18,Bundesautobahn,,Frankfurt (ots) (fue)
4,Festnahme eines Jugendlichen,Gestern (16.04.2019) hat ein 14-Jähriger glei...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-17,Innenstadt,NaN,Frankfurt (ots) (em)
